In [1]:
%matplotlib inline
import sys
import inspect
import matplotlib.pyplot as plt
import PIL
from pathlib import Path

here = !pwd
here = Path(here[0])

sys.path.insert(0,here.parent) 

from autocrop import Cropper

loc = 'data/macbeth.jpg'
c = Cropper()

ModuleNotFoundError: No module named 'autocrop'

In [ ]:
img_array = c.crop(loc)
PIL.Image.fromarray(img_array)

In [ ]:
c = Cropper()
img_array = c.crop(loc)
plt.imshow(img_array)
plt.show()

In [ ]:
%%timeit
# 32ms on 2016 i7 5-series, 148ms on 10 yo MBP
img_array = c.crop(loc)

In [ ]:
c = Cropper(face_percent=100)
plt.imshow(c.crop(loc))

In [ ]:
c = Cropper(height=700)
plt.imshow(c.crop(loc))

In [ ]:
noise = 'data/noise.png'
assert c.crop(noise) is None

In [ ]:
from glob import glob
faces = [f for f in glob('data/*') if not f.endswith('md')]

In [ ]:
print(img_array)

In [ ]:
def plot_test_images(faces, cropper):
    """Given a list on filepaths, crops and plots them."""
    for face in faces:
        try:
            img_array = c.crop(face)
        except (AttributeError, TypeError):
            pass
        if img_array is not None:
            plt.imshow(img_array)
            plt.show()

            
c = Cropper(width=650, height=500, face_percent=40)
plot_test_images(faces, c)

In [ ]:
c = Cropper(height=650, width=500, face_percent=40)
plot_test_images(faces, c)

## `_crop_postions`

In [ ]:
c = Cropper()
crop_positions = c._crop_positions
print(help(crop_positions))
crop_positions(500, 500, 50, 50, 100, 100)

In [ ]:
faces